<h3>Import các thư viện</h3>

In [36]:
from preprocess import Preprocessor # import các hàng tiền xử lí dữ liệu

#thư viện dùng cho đọc file
import pandas as pd # type: ignore

#thư viện làm việc với mảng
import numpy as np # type: ignore

#import model 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument # type: ignore
from gensim.utils import simple_preprocess # type: ignore
from gensim.models.phrases import Phrases, Phraser # type: ignore
from gensim.models import Word2Vec # type: ignore
# Bỏ qua tất cả các cảnh báo
import warnings
warnings.filterwarnings("ignore")

#import logger
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



<h3>Đọc dữ liệu</h3>

In [37]:
#lấy đường dẫn
relative_path = "../../resources/data/data.csv"
#đọc file csv
sample_df = pd.read_csv(relative_path)
df = sample_df # tạo copy

#đọc thử 5 mẫu dữ liệu đầu tiên
df.head()

,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,Custodia is not entitled to a Fed master accou...,The Federal Reserve does not have to give digi...,"March 29, 2024, 7:05PM EDT",Sarah Wynn,Policy,COURT HEARINGS-LAWSUITS,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,"Multicoin Capital's hedge fund has grown 9,281...",Multicoin Capital’s crypto-focused hedge fund ...,"March 29, 2024, 7:00PM EDT UPDATED: March 29, ...",Elizabeth Napolitano,Companies,INVESTMENT FIRMS,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1kx raises $75 million in latest funding round,"1kx has raised $75 million, the latest sign in...","March 29, 2024, 3:20PM EDT",Elizabeth Napolitano,Companies,NaN,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,CFTC Commissioner Pham says agency may be infr...,One of the Commodity Futures Trading Commissio...,"March 29, 2024, 12:06PM EDT",Sarah Wynn,Exchanges,CFTC-SEC,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,Bitcoin futures open interest reaches new high...,Open interest for bitcoin futures on centraliz...,"March 29, 2024, 11:03AM EDT UPDATED: March 29,...",Vishal Chawla,The Block,NaN,Bitcoin futures open interest on centralized e...,NaN


- <h5>Khảo sát định dạng các dữ liệu</h5>

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8334 entries, 0 to 8333
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Article link     8334 non-null   object 
 1    Website source  8334 non-null   object 
 2    Article type    8334 non-null   object 
 3    Article title   8334 non-null   object 
 4    Content         8331 non-null   object 
 5    Creation date   8334 non-null   object 
 6    Author          8333 non-null   object 
 7    Category        8334 non-null   object 
 8    Tags            5908 non-null   object 
 9    Summary         8129 non-null   object 
 10   Temp            0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 716.3+ KB


<h3>Tiền xử lý dữ liệu</h3>
<ul><li><h5>Tiền xử lý các trường text</h5></li><ul>

In [39]:
preprocessor = Preprocessor()

df[' Article title'] = df[' Article title'].apply(preprocessor.preprocess_text)
df[' Author'] = df[' Author'].apply(preprocessor.preprocess_text)
df[' Content'] = df[' Content'].apply(preprocessor.preprocess_text)
df[' Category'] = df[' Category'].apply(preprocessor.preprocess_text)
df[' Tags'] = df[' Tags'].apply(preprocessor.preprocess_text)

df.head(10)#đọc thử dữ liệu


,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,custodia entitle feed master account judge rule,federal reserve digital asset bank custodia ac...,"March 29, 2024, 7:05PM EDT",sarah wynn,policy,court hearing lawsuit,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,multicoin capital hedge fund grow 9281 2017 in...,multicoin capital crypto focus hedge fund brin...,"March 29, 2024, 7:00PM EDT UPDATED: March 29, ...",elizabeth napolitano,company,investment firm,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1 kx raise 75 million fund round,1 kx raise 75 million sign investor embrace cr...,"March 29, 2024, 3:20PM EDT",elizabeth napolitano,company,nan,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,cftc commissioner pham agency infringe sec aut...,commodity future trade commission leader carol...,"March 29, 2024, 12:06PM EDT",sarah wynn,exchange,cftc sec,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,bitcoin future reach 38 billion,bitcoin future centralize exchange reach fresh...,"March 29, 2024, 11:03AM EDT UPDATED: March 29,...",vishal chawla,block,nan,Bitcoin futures open interest on centralized e...,NaN
5,https://www.theblock.co/post/285659/grayscale-...,https://www.theblock.co,News Article,grayscale launch proof stake investment fund m...,grayscale announce launch dynamic income fund ...,"March 29, 2024, 10:57AM EDT",rt watson,fund,nan,Grayscale is launching a “dynamic income fund”...,NaN
6,https://www.theblock.co/post/285655/uk-judge-f...,https://www.theblock.co,News Article,uk court freeze 76 million worth craig wright ...,uk judge approve freeze 6 million 76 million w...,"March 29, 2024, 10:46AM EDT",mk manoylov,policy,bitcoin court hearing,A UK judge approved a so-called ‘worldwide fre...,NaN
7,https://www.theblock.co/post/285590/prismafi-h...,https://www.theblock.co,News Article,prisma hacker link wallet move fund tornado ca...,wallet connect hacker prisma finance 116 milli...,"March 29, 2024, 6:13AM EDT UPDATED: March 29, ...",danny park,security,exploit hack,Prisma Finance was hacked for over $11.6 milli...,NaN
8,https://www.theblock.co/post/285587/ethereum-v...,https://www.theblock.co,News Article,ethereum co founder vitalik buterin memecoins ...,blog post publish ethereum co founder vitalik ...,"March 29, 2024, 5:37AM EDT",adam james,crypto ecosystem,vitalik buterin,Vitalik Buterin published a new blog post on t...,NaN
9,https://www.theblock.co/post/285564/spot-bitco...,https://www.theblock.co,News Article,spot bitcoin etfs 4 consecutive day net inflow...,flow spot bitcoin exchange trade fund remain n...,"March 29, 2024, 4:03AM EDT",adam james,fund,nan,The daily total net inflow for the spot bitcoi...,NaN


- <h5>Tiền xử lý date</h5>

In [40]:
df[' Creation date'] = df[' Creation date'].apply(preprocessor.preprocess_date)

df.head(10)

,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,custodia entitle feed master account judge rule,federal reserve digital asset bank custodia ac...,2024-03-29,sarah wynn,policy,court hearing lawsuit,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,multicoin capital hedge fund grow 9281 2017 in...,multicoin capital crypto focus hedge fund brin...,2024-03-29,elizabeth napolitano,company,investment firm,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1 kx raise 75 million fund round,1 kx raise 75 million sign investor embrace cr...,2024-03-29,elizabeth napolitano,company,nan,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,cftc commissioner pham agency infringe sec aut...,commodity future trade commission leader carol...,2024-03-29,sarah wynn,exchange,cftc sec,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,bitcoin future reach 38 billion,bitcoin future centralize exchange reach fresh...,2024-03-29,vishal chawla,block,nan,Bitcoin futures open interest on centralized e...,NaN
5,https://www.theblock.co/post/285659/grayscale-...,https://www.theblock.co,News Article,grayscale launch proof stake investment fund m...,grayscale announce launch dynamic income fund ...,2024-03-29,rt watson,fund,nan,Grayscale is launching a “dynamic income fund”...,NaN
6,https://www.theblock.co/post/285655/uk-judge-f...,https://www.theblock.co,News Article,uk court freeze 76 million worth craig wright ...,uk judge approve freeze 6 million 76 million w...,2024-03-29,mk manoylov,policy,bitcoin court hearing,A UK judge approved a so-called ‘worldwide fre...,NaN
7,https://www.theblock.co/post/285590/prismafi-h...,https://www.theblock.co,News Article,prisma hacker link wallet move fund tornado ca...,wallet connect hacker prisma finance 116 milli...,2024-03-29,danny park,security,exploit hack,Prisma Finance was hacked for over $11.6 milli...,NaN
8,https://www.theblock.co/post/285587/ethereum-v...,https://www.theblock.co,News Article,ethereum co founder vitalik buterin memecoins ...,blog post publish ethereum co founder vitalik ...,2024-03-29,adam james,crypto ecosystem,vitalik buterin,Vitalik Buterin published a new blog post on t...,NaN
9,https://www.theblock.co/post/285564/spot-bitco...,https://www.theblock.co,News Article,spot bitcoin etfs 4 consecutive day net inflow...,flow spot bitcoin exchange trade fund remain n...,2024-03-29,adam james,fund,nan,The daily total net inflow for the spot bitcoi...,NaN


In [41]:
dropped_columns = ['Article link', ' Website source', ' Article type', ' Summary', ' Temp']
df.drop(columns=dropped_columns, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Article title,Content,Creation date,Author,Category,Tags
0,custodia entitle feed master account judge rule,federal reserve digital asset bank custodia ac...,2024-03-29,sarah wynn,policy,court hearing lawsuit
1,multicoin capital hedge fund grow 9281 2017 in...,multicoin capital crypto focus hedge fund brin...,2024-03-29,elizabeth napolitano,company,investment firm
2,1 kx raise 75 million fund round,1 kx raise 75 million sign investor embrace cr...,2024-03-29,elizabeth napolitano,company,nan
3,cftc commissioner pham agency infringe sec aut...,commodity future trade commission leader carol...,2024-03-29,sarah wynn,exchange,cftc sec
4,bitcoin future reach 38 billion,bitcoin future centralize exchange reach fresh...,2024-03-29,vishal chawla,block,nan
...,...,...,...,...,...,...
8329,binance launch 500 million fund provide loan b...,binance launch 500 million fund provide loan b...,2022-10-14,yogita khatri,mine,binance bitcoin mine
8330,kraken poach geminis head uk run uk operation,crypto exchange kraken poach geminis former he...,2022-10-14,tim copeland,hire,geminus hire kraken
8331,nydig lay 3 staff september wsj,bitcoin investment firm nydig lay 3 employee s...,2022-10-13,kristin majcher,bitcoin,layoff
8332,propose account rule benefit company hold crypto,financial account service board fasb unanimous...,2022-10-13,aislinn keely,regulation,microstrategy


In [42]:
output_file_path = "../../resources/data/preprocessed_data.csv"
df.to_csv(output_file_path)

<h3>Đánh trọng số cho các cột</h3>

In [43]:
column_weights = {
    ' Author': 1,
    ' Content': 1,
    ' Category': 1,
    ' Article title': 5,
    ' Tags': 1
}

<h3>Các hàm tokenize và tạo TaggedDocument</h3>

In [44]:
def tokenize_cell(cell): #tokenize 1 cell
    # Tokenize một từ bằng cách sử dụng Gensim
    return simple_preprocess(cell)

def tokenize_row(row, column_weights):
    sentences = []
    for column, weight in column_weights.items():
        if column in row:
            cell_tokens = tokenize_cell(row[column])
            sentences.extend([cell_tokens] * weight)
    return sentences



In [45]:
all_sentences = []
for index, row in df.iterrows():
    sentences = tokenize_row(row, column_weights)
    all_sentences.extend(sentences)

print(len(all_sentences))

75006


In [46]:
sent = [row.split() for row in df[' Content']]
phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
all_sentences = bigram[all_sentences]

2024-04-23 16:29:15,042 : INFO : collecting all words and their counts
2024-04-23 16:29:15,043 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-04-23 16:29:16,785 : INFO : collected 809740 token types (unigram + bigrams) from a corpus of 1623489 words and 8334 sentences
2024-04-23 16:29:16,785 : INFO : merged Phrases<809740 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
2024-04-23 16:29:16,786 : INFO : Phrases lifecycle event {'msg': 'built Phrases<809740 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 1.74s', 'datetime': '2024-04-23T16:29:16.786079', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2024-04-23 16:29:16,802 : INFO : exporting phrases from Phrases<809740 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
2024-04-23 16:29:18,084 : INFO : FrozenPhrases lifecycle event

In [47]:
def read_corpus(data, column_weights, bigram_model=None, tokens_only=False):
    
    if bigram_model is not None:
        for index, row in data.iterrows():
            tokens = []
            sentences = tokenize_row(row, column_weights)
            for sentence in sentences:
                if bigram_model:
                    sentence = bigram_model[sentence]
                tokens.extend(sentence)
            if tokens_only:
                yield tokens
            else:
                yield TaggedDocument(tokens, [index])
    else:
        for index, row in data.iterrows():
            sentences = tokenize_row(row, column_weights)
            if tokens_only:
                for sentence in sentences:
                    yield sentence
            else:
                for sentence in sentences:
                    yield TaggedDocument(sentence, [index])

<h3>Tạo TaggedDocuments và đọc thử</h3>

In [48]:
tagged_docs = list(read_corpus(data=df, column_weights=column_weights, bigram_model=bigram))
print(len(tagged_docs))
tagged_docs[:2]


8334


[TaggedDocument(words=['sarah', 'wynn', 'federal_reserve', 'digital_asset', 'bank', 'custodia', 'access', 'master', 'account', 'judge_rule', 'friday', 'federal_reserve', 'bank', 'discretion', 'grant', 'master', 'account', 'judge', 'scott', 'skavdahl', 'district_court', 'district', 'wyoming', 'judgment', 'news', 'blow', 'custodia', 'bank', 'sue', 'federal_reserve', 'board', 'governor', 'federal_reserve', 'bank', 'kansas', 'city', 'delay_decision', 'application', 'central_bank', 'master', 'account', 'master', 'account', 'allow', 'institution', 'direct', 'access', 'feds', 'payment_system', 'provide', 'direct', 'access', 'uss', 'money', 'supply', 'available', 'financial_institution', 'master', 'account', 'force', 'rely', 'partner', 'bank', 'master', 'account', 'provide', 'service', 'custodias', 'position', 'correct', 'effectively', 'mean', 'depository', 'institution', 'charter', 'law', 'regardless', 'soundly', 'craft', 'entitle', 'master', 'account', 'allow', 'direct', 'access', 'federal',

<h3>Train model</h3>

In [49]:
model = Doc2Vec(vector_size=50, window=5, min_count=1, epochs=50)
model.build_vocab(tagged_docs)
model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)


2024-04-23 16:29:22,710 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,s0.001,t3>', 'datetime': '2024-04-23T16:29:22.710324', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2024-04-23 16:29:22,718 : INFO : collecting all words and their counts
2024-04-23 16:29:22,719 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2024-04-23 16:29:23,035 : INFO : collected 31552 word types and 8334 unique tags from a corpus of 8334 examples and 1778427 words
2024-04-23 16:29:23,036 : INFO : Creating a fresh vocabulary
2024-04-23 16:29:23,129 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 31552 unique words (100.00% of original 31552, drops 0)', 'datetime': '2024-04-23T16:29:23.129239', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37

<h3>Thử tạo doc2vec với model</h3>

In [52]:
vector = model.infer_vector(['sam'], epochs=100, alpha=0.025)
vector2 = model.infer_vector(['sam', 'altman'], epochs=100, alpha=0.025)

print(vector)
print(vector2)

[-0.17652036 -0.15173292 -0.01846961  0.16694091  0.00830614 -0.27960345
  0.1826992   0.16811918 -0.4751811   0.04299874 -0.36114874 -0.33014005
  0.42722487 -0.19647156  0.27839258  0.31109414  0.4083051  -0.32579774
 -0.17492017 -0.23991878 -0.1176197  -0.16752316  0.42839482 -0.06985008
  0.3649103   0.21162488 -0.21620385 -0.1012274  -0.03581652 -0.11749282
  0.00524001  0.4508006  -0.07789199  0.11283731 -0.46224007 -0.0294871
 -0.25298434 -0.3241768   0.21932353  0.30551895  0.00157892 -0.02208306
 -0.17500901 -0.13124506  0.47474095  0.18165602 -0.25570804 -0.13247189
  0.17708212 -0.00804774]
[-0.35774925 -0.3505913   0.06032256  0.24990511 -0.02473193 -0.40080997
  0.13914712  0.25294134 -0.77168983  0.18457161 -0.68251073 -0.47066697
  0.7258449  -0.6709798   0.58530223  0.6216431   0.57148343 -0.3907388
 -0.37799653 -0.5430082  -0.26797542 -0.4348789   0.82841057 -0.34764656
  0.6293196   0.40229768 -0.34241226 -0.24235214 -0.15382694 -0.12979329
 -0.15366723  0.84491897 -0

In [56]:
model.wv.key_to_index

{'crypto': 0,
 'million': 1,
 'bitcoin': 2,
 'market': 3,
 'company': 4,
 'fund': 5,
 'exchange': 6,
 'trade': 7,
 'token': 8,
 'ftx': 9,
 'firm': 10,
 'sec': 11,
 'asset': 12,
 'user': 13,
 'billion': 14,
 'launch': 15,
 'report': 16,
 'accord': 17,
 'time': 18,
 'ethereum': 19,
 'include': 20,
 'price': 21,
 'network': 22,
 'add': 23,
 'platform': 24,
 'binance': 25,
 'blockchain': 26,
 'coinbase': 27,
 'base': 28,
 'datum': 29,
 'web': 30,
 'week': 31,
 'security': 32,
 'bank': 33,
 'project': 34,
 'raise': 35,
 'digital_asset': 36,
 'month': 37,
 'investor': 38,
 'lead': 39,
 'block': 40,
 'protocol': 41,
 'transaction': 42,
 'wallet': 43,
 'plan': 44,
 'investment': 45,
 'share': 46,
 'service': 47,
 'ceo': 48,
 'chain': 49,
 'offer': 50,
 'nft': 51,
 'follow': 52,
 'customer': 53,
 'increase': 54,
 'ecosystem': 55,
 'spot_bitcoin': 56,
 'continue': 57,
 'industry': 58,
 'ether': 59,
 'etf': 60,
 'team': 61,
 'support': 62,
 'provide': 63,
 'call': 64,
 'sell': 65,
 'day': 66,
 'p

<h3>Lưu model</h3>

In [51]:
model.save('models/doc2vec_model.model')

2024-04-23 16:30:42,995 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'models/doc2vec_model.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-04-23T16:30:42.995287', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'saving'}
2024-04-23 16:30:42,996 : INFO : not storing attribute cum_table
2024-04-23 16:30:43,018 : INFO : saved models/doc2vec_model.model
